In [1]:
import ticker_table
import pandas_datareader, re, sqlite3

In [2]:
class StockData:
    def __init__(self, se, sql):
        self.company_table = ticker_table.company_table(se = se, sql = sql)
        self.se = se
        self.sql_table = sql
   
    def lookup_ticker(self, company_name):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        result_table = []
        if not self.sql_table: 
            company_name = r'.*\s*'+company_name.lower()+r'\s.*'
            for row in self.company_table:
                match = re.search(company_name,row[1].lower()) 
                if match:
                    result_table.append((row[0],row[1]))
        else:
            conn = sqlite3.connect('data/companysql.sql')
            c = conn.cursor()
            query = ('%'+company_name+' %',)
            for row in c.execute('SELECT symbol,name FROM companysql WHERE lower(name) LIKE ?', query):
                result_table.append(row)
            conn.close()
        return result_table.pop() if len(result_table) == 1 else result_table

In [3]:
y = StockData('nasdaq', True)


In [4]:
y.lookup_ticker('intel')

('INTC', 'Intel Corporation')